In [1]:
%pip install dspy-ai==2.5.41 sglang[all] datasets huggingface

  Attempting uninstall: dspy-ai
    Found existing installation: dspy-ai 2.6.11
    Uninstalling dspy-ai-2.6.11:
      Successfully uninstalled dspy-ai-2.6.11
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install flashinfer -i https://flashinfer.ai/whl/cu121/torch2.4/ --force-reinstall

Looking in indexes: https://flashinfer.ai/whl/cu121/torch2.4/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.8/405.8 MB 39.2 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of flashinfer to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.8/405.8 MB 39.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 23.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: flashinfer
    Found existing installation: flashinfer 0.1.6+cu121torch2.4
    Uninstalling flashinfer-0.1.6+cu121torch2.4:
      Successfully uninstalled flashinfer-0.1.6+cu121torch2.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
PORT_NUMBER = 7501 # You can change the port number here
!CUDA_VISIBLE_DEVICES=0 python -m sglang.launch_server --port $PORT_NUMBER --model-path meta-llama/Llama-3.1-8B-Instruct

INFO 03-13 21:01:20 __init__.py:190] Automatically detected platform cuda.
Traceback (most recent call last):
  File "/home/coder/dev/hello-dspy/env/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/coder/dev/hello-dspy/env/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/coder/dev/hello-dspy/env/lib/python3.10/site-packages/sglang/launch_server.py", line 6, in <module>
    from sglang.srt.entrypoints.http_server import launch_server
  File "/home/coder/dev/hello-dspy/env/lib/python3.10/site-packages/sglang/srt/entrypoints/http_server.py", line 44, in <module>
    from sglang.srt.entrypoints.engine import _launch_subprocesses
  File "/home/coder/dev/hello-dspy/env/lib/python3.10/site-packages/sglang/srt/entrypoints/engine.py", line 36, in <module>
    from sglang.srt.managers.data_parallel_controller import (
  File "/home/coder/dev/hello-dspy/env/lib/python3.10/site-packages/

In [ ]:
import dspy

class CreateOnePrompt(dspy.Signature):
    """
    You are a helpful assistant that is very mindful of user privacy. You have access to a powerful large language model that you can query. Given a user request, create a prompt for your large language model that preserves user privacy, so that this model can help you complete the user request. Provide the prompt directly without any preamble. DO NOT COMPLETE THE USER QUERY, ONLY GENERATE A PROMPT.
    """
    userQuery = dspy.InputField(desc="The user's request to be fulfilled.")
    createdPrompt = dspy.OutputField()

class InfoAggregator(dspy.Signature):
    """
    You are a helpful assistant. Respond to queries from the user.
    """

    userQuery = dspy.InputField(desc="The user's request to be fulfilled.")
    modelExampleResponses = dspy.InputField(desc="Information from a more powerful language model responding to related queries. Complete the user query by referencing this information. Only you have access to this information.")
    finalOutput = dspy.OutputField()

class PAPILLON(dspy.Module):
    def __init__(self, untrusted_model):
        self.prompt_creater = dspy.ChainOfThought(CreateOnePrompt)
        self.info_aggregator = dspy.Predict(InfoAggregator)
        self.untrusted_model = untrusted_model

    def forward(self, user_query):
        try:
            prompt = self.prompt_creater(userQuery=user_query).createdPrompt
            response = self.untrusted_model(prompt)[0]
            output = self.info_aggregator(userQuery=user_query, modelExampleResponses=response)
        except Exception:
            return dspy.Prediction(prompt="", output="", gptResponse="")

        return dspy.Prediction(prompt=prompt, output=output.finalOutput, gptResponse=response)